Bank Churn Modelling Case Study

Column Description


RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,
NumOfProducts :How many accounts, bank account affiliated products the person has,
HasCrCard,IsActiveMember :Subjective but for the concept,EstimatedSalary,Exited:Did they leave the bank after all?

Problem Statement:

Given a Bank customer, can we build a classifier that can determine whether they will leave or not using Neural networks?

In [0]:
import tensorflow as tf

In [0]:
#!pip install -U tensorflow==2.0

In [2]:
tf.__version__

'2.0.0'

In [0]:
from google.colab import drive

In [4]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


1. Read the dataset

In [0]:
import numpy as np
import pandas as pd

In [0]:
churn=pd.read_csv('/content/drive/My Drive/Great Learning Residency 7.1 Neural Networks contd/project 1 neural networks/Churn.csv')

In [7]:
churn.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


2. Drop the columns which are unique for all users like IDs (2.5 points)

In [8]:
churn.shape 

(10000, 14)

In [9]:
churn['RowNumber'].nunique() # All unique values , should be dropped

10000

In [10]:
churn['CustomerId'].nunique() # All unique values , should be dropped

10000

In [11]:
churn['Surname'].nunique() # Not All unique values , could be dropped

2932

In [12]:
churn['Age'].nunique() # shouldn't be dropped

70

In [13]:
churn['Geography'].nunique() # shouldn't be dropped

3

In [14]:
churn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
RowNumber          10000 non-null int64
CustomerId         10000 non-null int64
Surname            10000 non-null object
CreditScore        10000 non-null int64
Geography          10000 non-null object
Gender             10000 non-null object
Age                10000 non-null int64
Tenure             10000 non-null int64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
Exited             10000 non-null int64
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [0]:
bank_churn = churn.drop(columns=['RowNumber','CustomerId','Surname'])

In [16]:
print ("Shape of the data: ", bank_churn.shape)
bank_churn.head()

Shape of the data:  (10000, 11)


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [17]:
bank_churn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
CreditScore        10000 non-null int64
Geography          10000 non-null object
Gender             10000 non-null object
Age                10000 non-null int64
Tenure             10000 non-null int64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
Exited             10000 non-null int64
dtypes: float64(2), int64(7), object(2)
memory usage: 859.5+ KB


In [0]:
# geography and gender are of object type

In [0]:
# Converting the Geography and Gender to categorial.
bank_churn["Geography"] = bank_churn["Geography"].astype('category')
bank_churn["Gender"] = bank_churn["Gender"].astype('category')

In [20]:
bank_churn["Geography"] = bank_churn["Geography"].cat.codes
bank_churn["Gender"] = bank_churn["Gender"].cat.codes

bank_churn.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,0,0,42,2,0.00,1,1,1,101348.88,1
1,608,2,0,41,1,83807.86,1,0,1,112542.58,0
2,502,0,0,42,8,159660.80,3,1,0,113931.57,1
3,699,0,0,39,1,0.00,2,0,0,93826.63,0
4,850,2,0,43,2,125510.82,1,1,1,79084.10,0


3. Distinguish the feature and target set (2.5 points)

In [0]:
# Excited is the target column

In [0]:
Target_set=pd.DataFrame(bank_churn['Exited'])
Feature_set=bank_churn.drop(['Exited'],axis=1)

In [23]:
Target_set.head()

,Exited
0,1
1,0
2,1
3,0
4,0


In [24]:
Feature_set.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,0,0,42,2,0.00,1,1,1,101348.88
1,608,2,0,41,1,83807.86,1,0,1,112542.58
2,502,0,0,42,8,159660.80,3,1,0,113931.57
3,699,0,0,39,1,0.00,2,0,0,93826.63
4,850,2,0,43,2,125510.82,1,1,1,79084.10


4. Divide the data set into train and test sets

In [0]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(Feature_set,Target_set,test_size=0.2)

In [26]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(8000, 10) (2000, 10) (8000, 1) (2000, 1)


In [27]:
X_train.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
2584,748,0,1,40,0,0.00,1,0,0,60416.76
7067,746,0,0,24,4,0.00,1,0,1,94105.00
8288,850,0,0,34,9,92899.27,2,1,0,97465.89
7437,628,0,0,29,9,71996.29,1,1,1,34857.46
4587,500,0,0,34,6,0.00,1,1,1,140268.45


In [28]:
X_test.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
1601,834,0,0,49,8,160602.25,2,1,0,129273.94
9297,737,0,1,30,8,174356.13,1,0,0,31928.50
3226,603,1,1,35,1,105346.03,2,1,1,130379.50
744,650,0,1,60,8,0.00,2,1,1,102925.76
8573,633,0,0,26,8,124281.84,1,1,1,60116.57


In [29]:
y_test.head()

,Exited
1601,0
9297,0
3226,0
744,0
8573,0


In [30]:
y_train.head()

,Exited
2584,0
7067,0
8288,0
7437,0
4587,0


In [31]:
print ("Unique train labels: ", np.unique(y_train))

Unique train labels:  [0 1]


In [32]:
print ("Unique test labels: ", np.unique(y_test))

Unique test labels:  [0 1]


5. Normalize the train and test data (2.5 points)

In [0]:
from scipy import stats

X_train_std = stats.zscore(X_train) 
X_test_std = stats.zscore(X_test)

In [34]:
X_test_std

array([[ 1.91227945, -0.91512816, -1.12017473, ...,  0.64686916,
        -1.07047781,  0.54251572],
       [ 0.89678569, -0.91512816,  0.89271787, ..., -1.54590766,
        -1.07047781, -1.14378577],
       [-0.50606137,  0.29535883,  0.89271787, ...,  0.64686916,
         0.93416229,  0.56166718],
       ...,
       [ 0.2477072 , -0.91512816, -1.12017473, ..., -1.54590766,
        -1.07047781,  0.8362541 ],
       [ 0.5199014 , -0.91512816, -1.12017473, ...,  0.64686916,
        -1.07047781,  0.95944996],
       [-0.82013161,  1.50584582,  0.89271787, ..., -1.54590766,
         0.93416229,  1.00401925]])

In [35]:
X_train_std

array([[ 1.00769944, -0.8986023 ,  0.91739884, ..., -1.5482337 ,
        -1.02096982, -0.69998064],
       [ 0.98706545, -0.8986023 , -1.09003844, ..., -1.5482337 ,
         0.97946088, -0.1134883 ],
       [ 2.06003277, -0.8986023 , -1.09003844, ...,  0.64589732,
        -1.02096982, -0.0549772 ],
       ...,
       [-1.42711102,  0.30939945,  0.91739884, ..., -1.5482337 ,
         0.97946088, -1.09999418],
       [-1.30330709, -0.8986023 ,  0.91739884, ...,  0.64589732,
        -1.02096982, -0.45644773],
       [-1.26203912,  0.30939945, -1.09003844, ...,  0.64589732,
        -1.02096982, -0.89379541]])

In [0]:
y_train_cat = tf.keras.utils.to_categorical(y_train)
y_test_cat = tf.keras.utils.to_categorical(y_test)

In [37]:
y_train_cat

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [0., 1.],
       [1., 0.],
       [1., 0.]], dtype=float32)

6. Initialize & build the model (10 points)




In [0]:
#Clear out tensorflow memory
tf.keras.backend.clear_session()

#Initialize Sequential model
model = tf.keras.models.Sequential()

#Normalize the data
model.add(tf.keras.layers.BatchNormalization())

In [0]:
#Adding 1st dence layer
model.add(tf.keras.layers.Dense(20, activation='sigmoid',input_shape=(10,))) # we have 10 features so 10 input_shape

#Add 2nd hidden layer
model.add(tf.keras.layers.Dense(10, activation='sigmoid')) 

#Adding OUTPUT layer
model.add(tf.keras.layers.Dense(2, activation='softmax')) # outputs are 2

In [0]:
#Comile the model
model.compile(optimizer='sgd', loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [41]:
model.fit(X_train_std, y_train_cat, 
          validation_data=(X_test_std, y_test_cat), 
          epochs=10,
          batch_size=32)

Train on 8000 samples, validate on 2000 samples
Epoch 1/10
8000/8000 [==============================] - 1s 166us/sample - loss: 0.5162 - accuracy: 0.7958 - val_loss: 0.5073 - val_accuracy: 0.7985
Epoch 2/10
8000/8000 [==============================] - 0s 61us/sample - loss: 0.5105 - accuracy: 0.7958 - val_loss: 0.5041 - val_accuracy: 0.7985
Epoch 3/10
8000/8000 [==============================] - 0s 62us/sample - loss: 0.5071 - accuracy: 0.7958 - val_loss: 0.5012 - val_accuracy: 0.7985
Epoch 4/10
8000/8000 [==============================] - 1s 63us/sample - loss: 0.5040 - accuracy: 0.7958 - val_loss: 0.4984 - val_accuracy: 0.7985
Epoch 5/10
8000/8000 [==============================] - 1s 66us/sample - loss: 0.5006 - accuracy: 0.7958 - val_loss: 0.4955 - val_accuracy: 0.7985
Epoch 6/10
8000/8000 [==============================] - 1s 63us/sample - loss: 0.4976 - accuracy: 0.7958 - val_loss: 0.4927 - val_accuracy: 0.7985
Epoch 7/10
8000/8000 [==============================] - 1s 63us/sampl

In [0]:
# can optimise this model by using the learning rate parameter

7. Optimize the model (5 points)

In [0]:
#Compile the model
sgd_optimizer = tf.keras.optimizers.SGD(lr=0.03)
model.compile(optimizer=sgd_optimizer,  loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [43]:
model.fit(X_train_std, y_train_cat, 
          validation_data=(X_test_std, y_test_cat), 
          epochs=10, batch_size=32)

Train on 8000 samples, validate on 2000 samples
Epoch 1/10
8000/8000 [==============================] - 1s 121us/sample - loss: 0.4756 - accuracy: 0.7958 - val_loss: 0.4690 - val_accuracy: 0.7985
Epoch 2/10
8000/8000 [==============================] - 1s 64us/sample - loss: 0.4630 - accuracy: 0.7958 - val_loss: 0.4584 - val_accuracy: 0.7985
Epoch 3/10
8000/8000 [==============================] - 0s 61us/sample - loss: 0.4516 - accuracy: 0.7964 - val_loss: 0.4504 - val_accuracy: 0.7975
Epoch 4/10
8000/8000 [==============================] - 0s 62us/sample - loss: 0.4422 - accuracy: 0.7961 - val_loss: 0.4455 - val_accuracy: 0.7900
Epoch 5/10
8000/8000 [==============================] - 1s 64us/sample - loss: 0.4386 - accuracy: 0.7972 - val_loss: 0.4430 - val_accuracy: 0.7915
Epoch 6/10
8000/8000 [==============================] - 1s 63us/sample - loss: 0.4354 - accuracy: 0.8015 - val_loss: 0.4415 - val_accuracy: 0.7900
Epoch 7/10
8000/8000 [==============================] - 1s 63us/sampl

In [0]:
# Accuracy has increased slightly

In [0]:
# Building another model with input activation function as 'relu' and loss function as 'binary_crossentropy' as we have two output labels

In [0]:
#Clear out tensorflow memory
tf.keras.backend.clear_session()

#Initialize Sequential model
model1 = tf.keras.models.Sequential()

#Input Layer
model1.add(tf.keras.layers.Dense(20, activation='relu',input_shape=(10,)))

#Add 2nd hidden layer
model1.add(tf.keras.layers.Dense(10, activation='relu')) 

#Add Dense Layer which provides 1 Output after applying sigmoid (Output Layer)
model1.add(tf.keras.layers.Dense(2, activation='softmax'))



In [0]:
#Comile the model
model1.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])

In [48]:
model1.fit(X_train_std, y_train_cat, 
          validation_data=(X_test_std, y_test_cat), 
          epochs=10,
          batch_size=10)

Train on 8000 samples, validate on 2000 samples
Epoch 1/10
8000/8000 [==============================] - 2s 249us/sample - loss: 0.4959 - accuracy: 0.7879 - val_loss: 0.4672 - val_accuracy: 0.7990
Epoch 2/10
8000/8000 [==============================] - 2s 189us/sample - loss: 0.4411 - accuracy: 0.7974 - val_loss: 0.4444 - val_accuracy: 0.8035
Epoch 3/10
8000/8000 [==============================] - 2s 189us/sample - loss: 0.4203 - accuracy: 0.8114 - val_loss: 0.4287 - val_accuracy: 0.8225
Epoch 4/10
8000/8000 [==============================] - 2s 189us/sample - loss: 0.4021 - accuracy: 0.8340 - val_loss: 0.4132 - val_accuracy: 0.8350
Epoch 5/10
8000/8000 [==============================] - 2s 188us/sample - loss: 0.3859 - accuracy: 0.8462 - val_loss: 0.3988 - val_accuracy: 0.8425
Epoch 6/10
8000/8000 [==============================] - 2s 194us/sample - loss: 0.3725 - accuracy: 0.8503 - val_loss: 0.3838 - val_accuracy: 0.8460
Epoch 7/10
8000/8000 [==============================] - 2s 189us

In [0]:
# Compiling the above model with 'adam' as the optimizer

In [0]:
#Comile the model
model1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [51]:
model1.fit(X_train_std, y_train_cat, 
          validation_data=(X_test_std, y_test_cat), 
          epochs=10,
          batch_size=10)

Train on 8000 samples, validate on 2000 samples
Epoch 1/10
8000/8000 [==============================] - 2s 261us/sample - loss: 0.3469 - accuracy: 0.8564 - val_loss: 0.3575 - val_accuracy: 0.8525
Epoch 2/10
8000/8000 [==============================] - 2s 194us/sample - loss: 0.3446 - accuracy: 0.8604 - val_loss: 0.3617 - val_accuracy: 0.8510
Epoch 3/10
8000/8000 [==============================] - 2s 192us/sample - loss: 0.3420 - accuracy: 0.8584 - val_loss: 0.3584 - val_accuracy: 0.8525
Epoch 4/10
8000/8000 [==============================] - 2s 190us/sample - loss: 0.3391 - accuracy: 0.8608 - val_loss: 0.3593 - val_accuracy: 0.8500
Epoch 5/10
8000/8000 [==============================] - 2s 190us/sample - loss: 0.3370 - accuracy: 0.8609 - val_loss: 0.3547 - val_accuracy: 0.8530
Epoch 6/10
8000/8000 [==============================] - 2s 190us/sample - loss: 0.3362 - accuracy: 0.8630 - val_loss: 0.3511 - val_accuracy: 0.8510
Epoch 7/10
8000/8000 [==============================] - 2s 194us

In [0]:
# accuracy has further improved

9. Predict the results using 0.5 as a threshold (5 points)

In [53]:
model1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 20)                220       
_________________________________________________________________
dense_1 (Dense)              (None, 10)                210       
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 22        
Total params: 452
Trainable params: 452
Non-trainable params: 0
_________________________________________________________________


In [54]:
#Predicting the Test set results
y_pred = model1.predict(X_test_std)
#Threshold of 50%
y_pred = (y_pred > 0.5)
y_pred[:5]

array([[ True, False],
       [ True, False],
       [ True, False],
       [ True, False],
       [ True, False]])

10. Print the Accuracy score and confusion matrix (2.5 points)

In [0]:
from sklearn.metrics import accuracy_score
acc=accuracy_score(y_test_cat, y_pred)

In [56]:
print('accuracy=',acc*100)

accuracy= 85.39999999999999


In [0]:
from sklearn import metrics

In [59]:
print ("Confusion Matrix for predictions with specified threshold 0.5")
pd.DataFrame(metrics.confusion_matrix(y_test_cat.argmax(axis=1), y_pred.argmax(axis=1)),
                 columns=['pred_neg', 'pred_pos'], index=['neg', 'pos'])

Confusion Matrix for predictions with specified threshold 0.5


,pred_neg,pred_pos
neg,1524,73
pos,219,184
